In [1]:
pip install pgmpy pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from pgmpy.models import DiscreteBayesianNetwork 
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
import itertools

In [3]:
#Definir la estructura
model = DiscreteBayesianNetwork([
    ('Precio', 'Compra'),
    ('Genero', 'Compra'),
    ('Criticas', 'Compra'),
    ('Plataforma', 'Compra'),
    ('Amigos', 'Compra')
])

In [4]:
#definir los estados de variables
states = {
    'Precio': ['Alto', 'Medio', 'Bajo'],
    'Genero': ['Accion', 'Aventura', 'Estrategia', 'Otros'],
    'Criticas': ['Alta', 'Baja'],
    'Plataforma': ['Consola', 'PC'],
    'Amigos': ['Si', 'No'],
    'Compra': ['Si', 'No']
}

In [5]:
cpd_precio = TabularCPD(variable='Precio', variable_card=3, values=[[0.3], [0.4], [0.3]], state_names=states)
cpd_genero = TabularCPD(variable='Genero', variable_card=4, values=[[0.4], [0.3], [0.2], [0.1]], state_names=states)
cpd_criticas = TabularCPD(variable='Criticas', variable_card=2, values=[[0.6], [0.4]], state_names=states)
cpd_plataforma = TabularCPD(variable='Plataforma', variable_card=2, values=[[0.5], [0.5]], state_names=states)
cpd_amigos = TabularCPD(variable='Amigos', variable_card=2, values=[[0.4], [0.6]], state_names=states)


parent_vars = ['Precio', 'Genero', 'Criticas', 'Plataforma', 'Amigos']
cardinalities = [3, 4, 2, 2, 2]


combinations = list(itertools.product(
    states['Precio'], states['Genero'], states['Criticas'], 
    states['Plataforma'], states['Amigos']
))

probs_si = []
for comb in combinations:
    p, g, c, pl, a = comb
    score = 0.5
    if p == 'Bajo': score += 0.3
    elif p == 'Alto': score -= 0.3
    if c == 'Alta': score += 0.2
    if a == 'Si': score += 0.2
    score = max(0.01, min(0.99, score))
    probs_si.append(score)

probs_no = [1 - p for p in probs_si]
values_flat = [probs_si, probs_no] 


cpd_compra = TabularCPD(
    variable='Compra', 
    variable_card=2, 
    values=values_flat,
    evidence=parent_vars,
    evidence_card=cardinalities,
    state_names=states
)

In [6]:
model.add_cpds(cpd_precio, cpd_genero, cpd_criticas, cpd_plataforma, cpd_amigos, cpd_compra)


print("¿Modelo válido?", model.check_model())

infer = VariableElimination(model)
caso = {'Precio': 'Alto', 'Criticas': 'Alta', 'Amigos': 'Si'}
print(f"\nCalculando probabilidad para: {caso}")
print(infer.query(variables=['Compra'], evidence=caso))

¿Modelo válido? True

Calculando probabilidad para: {'Precio': 'Alto', 'Criticas': 'Alta', 'Amigos': 'Si'}
+------------+---------------+
| Compra     |   phi(Compra) |
+============+===============+
| Compra(Si) |        0.6000 |
+------------+---------------+
| Compra(No) |        0.4000 |
+------------+---------------+
